# FIN 514 - Project 2 Choice #1 Python Codes_CN
**Spring 2022**

This notebook provides the valuation of Auto Callable Yield Notes Linked to the S&P500® Index due July 21, 2023
Fully and Unconditionally Guaranteed by JPMorgan Chase & Co.

Team members: Ya-Yen,Li(yayenli2) / Wei Ting,Chao(wtchao3) / Yu-Shiuan,Chang(yschang4)

### Part a) Estimated value from Crank-Nicolson Method

### Part b) Volatility Sensitive Analysis

### Part c) Non-linearity Analysis

This notebook provides a simple Explicit Finite Difference Code to value European style options


## Packages and Configurations

The following common packages will be use on this notebook.
* numpy - [https://numpy.org/](https://numpy.org/)
* Pandas - [https://pandas.pydata.org/](https://pandas.pydata.org/)
* matplotlib - [https://matplotlib.org/](https://matplotlib.org/)
* Scipy Statistical functions - [https://docs.scipy.org/doc/scipy/reference/stats.html](https://docs.scipy.org/doc/scipy/reference/stats.html)

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

# Part a) Estimated value from Crank-Nicolson Method

## Step 1: Set the parameters

In [2]:
jmax = 250        # Number of S steps
imax = 10000      # Number of t steps
S0 = 4577.1       # Original Stock Price (1/18/2022)
sigma = 0.21589   # Annualized (Future) Volatility of Stock Price Returns
r = 0.007555      # Annualized Continously Compounded Risk-free Rate
q = 0.01455       # Continuous Dividend Yield
TM = 549/365      # Time of Maturity Date (7/21/2023)
TFR = 546/365     # Time of Final Review Date (7/18/2023)
SL = 0            # Minimum S value
SU = 2.5*S0       # Maximum S value
CPN = 5.375       # Coupon Payements per month
Trigger = 0.7*S0  # Trigger Barrier

In [3]:
# Review dates for autocall (Final Review Date TFM included)
Review_dates = [90/365, 120/365, 154/365, 181/365, 212/365, 244/365, 273/365, 304/365,
                335/365, 365/365, 399/365, 426/365, 455/365, 485/365, 518/365, 546/365]  

# Payment dates for coupons from T3-T17 (Final Review Date TM NOT included) 
Payment_dates = [93/365, 125/365, 157/365, 184/365, 217/365, 247/365, 276/365, 309/365, 
                 338/365, 370/365,402/365, 429/365, 458/365, 490/365, 521/365]  

# Payment dates for coupons from T1,T2 (no autocall)
CPN_payment_dates = [37/365, 64/365]

In [4]:
# Check for dates

# delta t
dt = TFR / imax 
print(f"dt: {dt}")

# Review dates for autocall (Final Review Date TM included)
ireview1 = [i/dt for i in Review_dates]      
ireview = [int(i) for i in ireview1]     
print(f"ireview:   {ireview}")

# Payment dates for coupons from T3~T17 (Final Review Date TM NOT included) 
ipayment1 = [i/dt for i in Payment_dates]  
ipayment = [int(i) for i in ipayment1]
print(f"ipayment1: {ipayment}")

# Payment dates for coupons from T1,T2 (coupon only date)
icpn1 = [i/dt for i in CPN_payment_dates]      
icpn = [int(i) for i in icpn1]     
print(f"icpn1:     {icpn}")

# Payment dates for all coupons from T1~T17 
itotal = icpn + ipayment
print(f"itotal:    {itotal}")

dt: 0.0001495890410958904
ireview:   [1648, 2197, 2820, 3315, 3882, 4468, 5000, 5567, 6135, 6684, 7307, 7802, 8333, 8882, 9487, 10000]
ipayment1: [1703, 2289, 2875, 3369, 3974, 4523, 5054, 5659, 6190, 6776, 7362, 7857, 8388, 8974, 9542]
icpn1:     [677, 1172]
itotal:    [677, 1172, 1703, 2289, 2875, 3369, 3974, 4523, 5054, 5659, 6190, 6776, 7362, 7857, 8388, 8974, 9542]


## Step 2: Build the Crank-Nicolson method function

In [5]:
def CNFD(S0, Trigger, TM, TFR, r, q, sigma, SU, imax, jmax, CPN, Review_dates, Payment_dates, CPN_payment_dates):
    
    # CREATE TWO DIMENSIONAL ARRAY OF SIZE [imax+1,jmax+1] TO STORE V AND VT AT ALL STEPS   
    V = np.zeros([imax+1, jmax+1])
    VT = np.zeros([imax+1, jmax+1])

    # CREATE ONE DIMENSIONAL ARRAY OF SIZE [jmax+1] TO STORE A, B, C VALUES AT ALL STEPS
    # A[jmax], B[jmax], C[jmax], alpha[jmax], CN_S[jmax]
    A = np.zeros([jmax+1])
    B = np.zeros([jmax+1])
    C = np.zeros([jmax+1])
    D = np.zeros([jmax+1])
    alpha = np.zeros([jmax+1])
    CN_S = np.zeros([jmax+1])
    
    # Set up time and S steps
    dt = TFR / imax
    dS = SU  / jmax
    
    # Set up Autocall and Trigger steps 
    j0 = int(S0/dS)
    jb = int(Trigger/dS)
    
    # Review dates for autocall (Final Review Date TM included)
    ireview1 = [i/dt for i in Review_dates]       
    ireview = [int(i) for i in ireview1]    

    # Payment dates for coupons T3~T17 (Final Review Date TM NOT included) 
    ipayment1 = [i/dt for i in Payment_dates]      
    ipayment = [int(i) for i in ipayment1]    
    
    # Payment dates for coupons T1,T2 (coupon only date)
    icpn1 = [i/dt for i in CPN_payment_dates]        
    icpn = [int(i) for i in icpn1]     
    
    # Payment dates for all coupons T1~T17
    itotal = icpn + ipayment
    
    ### Grid for VT (Trigger event DOES happan) ###########################################################
    
    ### At maturity
    # Calculate option value at maturity V[imax,j]
    # Also calculate the probabilities A, B, C
    i = imax 
        
    for j in range(0, jmax+1):    

        if j >= j0:  # Autocall 
            VT[i, j] = (1000 + CPN) *  np.exp(-r *(TM - TFR))  
        else:
            VT[i, j] = (1000 * (j*dS/S0) + CPN) *  np.exp(-r *(TM - TFR))
        
        A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
        B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
        C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j
    
    ### Backward in time
    counter1 = 0
    counter2 = 0
    
    for i in range(imax-1, -1, -1):

        ### Lower boundary condition in matrix terms
        A[0] = 0
        B[0] = 1
        C[0] = 0
        D[0] = 0
        
        # Check if i meet the review dates (T18 not included)
        # D[0]=PV(remaining coupons)
        if (i+1) in itotal:     
            counter1 += 1
        if counter1 > 0:
            PV_cpn = []         # List to save the dicounted coupons 
            for k in range(1, counter1 +1):  
                PV_cpn.append(CPN * np.exp(-r *(itotal[-k] * dt - i * dt)))  # Append dicounted coupons in the list
            D[0] = sum(PV_cpn)  # Sum up the dicounted coupons 

        # Regular D[j] values
        # Exclude D[0], D[jmax]
        for j in range(1, jmax, 1):  
            D1 = -VT[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
            D2 = -VT[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
            D3 = -VT[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
            D[j] = D1+D2+D3

        ### Upper boundary condition in matrix terms
        A[jmax] = 0
        B[jmax] = 1
        C[jmax] = 0
        if (i+1) in ireview:
            counter2 += 1
        D[jmax] = (1000 + CPN) * np.exp(-r * (ireview[-counter2] * dt - i * dt))
        # ireview[-counter2]: time of next autocall
        
        ### Start LU decomposition
        alpha[0] = B[0]
        CN_S[0]  = D[0]
        
        # Where to stop solving?
        if i in ireview:         # Autocall dates
            StopFactor = j0+1    # Stop at j0 (Solve from 0~j0)
        else:                   
            StopFactor = jmax+1  # Stop at jmax (Solve from 0~jmax)
        
        # Solve alpha, CN_S
        for j in range(1, StopFactor, 1):
            alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
            CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
        VT[i,(StopFactor - 1)] = CN_S[(StopFactor - 1)]/alpha[(StopFactor - 1)]  
        
        # But at autocall dates, boundaries are different
        # A[j0] = 0, B[j0] = 1, C[j0] = 0, so change VT[i,j0:jmax+1] to D[j0]
        if i in ireview:           
            VT[i,j0:jmax+1] = (1000 + CPN) * np.exp(-r * (ipayment[ireview.index(i)] * dt - i * dt))
            # ipayment[ireview.index(i)]: time of coupon date
        
        # Start from j0-1 or jmax-1 to j0
        for j in range(StopFactor-2,-1,-1):
            VT[i,j] = (CN_S[j]-C[j]*VT[i,j+1])/alpha[j]
            if i in icpn:     # Payment dates for coupons T1,T2 (coupon only date) 
                VT[i,j] += CPN
            if i in ireview:  # Payment dates for coupons T3~T17
                VT[i,j] += CPN * np.exp(-r * (ipayment[ireview.index(i)] * dt - i * dt))      
            
    ### Grid for V (Trigger event does NOT happan) ###########################################################
    
    ### At maturity
    # Calculate option value at maturity V[imax,j]
    # Also calculate the probabilities A, B, C
    i = imax 
    
    for j in range(jb, jmax+1):  
        V[i, j] = (1000 + CPN) *  np.exp(-r *(TM - TFR))  
        A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
        B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
        C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j
    
    ### Backward in time
    counter3 = 0
    
    for i in range(imax-1, -1, -1):

        # Lower boundary condition for V
        A[jb] = 0
        B[jb] = 1
        C[jb] = 0
        
        # Get value from VT
        D[jb] = VT[i, jb] 

        # Regular D values
        for j in range(jb+1, jmax, 1):
            D1 = -V[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
            D2 = -V[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
            D3 = -V[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
            D[j] = D1+D2+D3
            
        # Upper boundary condition for V
        A[jmax] = 0
        B[jmax] = 1
        C[jmax] = 0
        if (i+1) in ireview:
            counter3 += 1
        D[jmax] = (1000 + CPN) * np.exp(-r * (ireview[-counter3] * dt - i * dt))
        # ireview[-counter3]: time of next autocall

        ### Start LU decomposition
        alpha[jb] = B[jb]
        CN_S[jb] = D[jb]
        
        # Where to stop solving?
        if i in ireview:         # Autocall dates
            StopFactor = j0+1    # Stop at j0 (Solve from 0~j0)
        else:
            StopFactor = jmax+1  # Stop at jmax (Solve from 0~jmax)
        
        # Solve alpha, CN_S
        for j in range(jb+1, StopFactor, 1):
            alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
            CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
        V[i,(StopFactor - 1)] = CN_S[(StopFactor - 1)]/alpha[(StopFactor - 1)]
        
        # But at autocall dates, boundaries are different
        # A[j0] = 0, B[j0] = 1, C[j0] = 0, so change V[i,j0:jmax+1] to D[j0]
        if i in ireview:          
            V[i,j0:jmax+1] = (1000 + CPN) * np.exp(-r * (ipayment[ireview.index(i)] * dt - i * dt))
            # ipayment[ireview.index(i)]: time of coupon date
            
        # Start from j0-1 or jmax-1
        for j in range(StopFactor-2,jb-1,-1):
            V[i,j] = (CN_S[j]-C[j]*V[i,j+1])/alpha[j]
            if i in icpn:      # Payment dates for coupons T1,T2 (coupon only date) 
                V[i,j] += CPN
            if i in ireview:   # Payment dates for coupons T3~T17
                V[i,j] += CPN * np.exp(-r * (ipayment[ireview.index(i)] * dt - i * dt))                      
    
    # If value lower than LBC(jb), then get value from VT
    V[:, 0:jb] = VT[:, 0:jb]

    # Final option value V[0, j0]
    return V[0, j0]


In [6]:
cnfd = CNFD(S0, Trigger, TM, TFR, r, q, sigma, SU, imax, jmax, CPN, Review_dates, Payment_dates, CPN_payment_dates)
cnfd

991.7550127655967

In [7]:
cnfd_1 = CNFD(S0, Trigger, TM, TFR, r, q, 0.24, SU, imax, jmax, CPN, Review_dates, Payment_dates, CPN_payment_dates)
cnfd_1

982.684652905072

# Part b) Volatility Sensitive Analysis

In [8]:
# Volatities from obersavation dates 
vol = [0.3915633, 0.2627887,  # 2022/05/03
       0.3625552, 0.2603371,  # 2022/11/01
       0.3508948, 0.2762897,  # 2022/08/02
       0.3383119, 0.2713139]  # 2023/01/31

In [9]:
# List to save the value 
value_result = []

# Value with different volatilities
for sigma in vol:
    note_value = CNFD(S0, Trigger, TM, TFR, r, q, sigma, SU, imax, jmax, CPN, Review_dates, Payment_dates, CPN_payment_dates)
    output = {'Volatility': sigma, 'Value': note_value}
    value_result.append(output)

In [10]:
# Print value with different volatilities
value_result

[{'Volatility': 0.3915633, 'Value': 940.4016304302024},
 {'Volatility': 0.2627887, 'Value': 974.9272588786571},
 {'Volatility': 0.3625552, 'Value': 947.4020317606729},
 {'Volatility': 0.2603371, 'Value': 975.7258739175453},
 {'Volatility': 0.3508948, 'Value': 950.2872113855633},
 {'Volatility': 0.2762897, 'Value': 970.6709938672059},
 {'Volatility': 0.3383119, 'Value': 953.4631998378939},
 {'Volatility': 0.2713139, 'Value': 972.2126338277358}]

# Part c) Non-linearity Analysis

In [11]:
def CNFD_1(S0, Trigger, TM, TFR, r, q, sigma, SU, imax, jmaxmin, jmaxmax, jmaxstep, CPN, Review_dates, Payment_dates, CPN_payment_dates):
    
    # LIST TO SAVE RESULTS
    cnfdb_result = []
    
    # Test different jmax
    for jmax in range(jmaxmin, jmaxmax, jmaxstep):
    
        # CREATE TWO DIMENSIONAL ARRAY OF SIZE [imax+1,jmax+1] TO STORE V AND VT AT ALL STEPS   
        V = np.zeros([imax+1, jmax+1])
        VT = np.zeros([imax+1, jmax+1])

        # CREATE ONE DIMENSIONAL ARRAY OF SIZE [jmax+1] TO STORE A, B, C VALUES AT ALL STEPS
        # A[jmax], B[jmax], C[jmax] 
        A = np.zeros([jmax+1])
        B = np.zeros([jmax+1])
        C = np.zeros([jmax+1])
        D = np.zeros([jmax+1])
        alpha = np.zeros([jmax+1])
        CN_S = np.zeros([jmax+1])

        # Set up time and S steps
        dt = TFR / imax
        dS = SU  / jmax

        # Set up Autocall and Trigger steps 
        j0 = int(S0/dS)
        jb = int(Trigger/dS)

        # Review dates for autocall (Final Review Date TM included)
        ireview1 = [i/dt for i in Review_dates]       
        ireview = [int(i) for i in ireview1]    

        # Payment dates for coupons T3~T17 (Final Review Date TM NOT included) 
        ipayment1 = [i/dt for i in Payment_dates]      
        ipayment = [int(i) for i in ipayment1]    

        # Payment dates for coupons T1,T2 (coupon only date)
        icpn1 = [i/dt for i in CPN_payment_dates]        
        icpn = [int(i) for i in icpn1]     

        # Payment dates for all coupons T1~T17
        itotal = icpn + ipayment

        ### Grid for VT (Trigger event DOES happan) ###########################################################

        ### At maturity
        # Calculate option value at maturity V[imax,j]
        # Also calculate the probabilities A, B, C
        i = imax 

        for j in range(0, jmax+1):    

            if j >= j0:  # Autocall 
                VT[i, j] = (1000 + CPN) *  np.exp(-r *(TM - TFR))  
            else:
                VT[i, j] = (1000 * (j*dS/S0) + CPN) *  np.exp(-r *(TM - TFR))

            A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
            B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
            C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j

        ### Backward in time
        counter1 = 0
        counter2 = 0

        for i in range(imax-1, -1, -1):

            ### Lower boundary condition in matrix terms
            A[0] = 0
            B[0] = 1
            C[0] = 0
            D[0] = 0

            # Check if i meet the review dates (T18 not included)
            # D[0]=PV(remaining coupons)
            if (i+1) in itotal:     
                counter1 += 1
            if counter1 > 0:
                PV_cpn = []         # List to save the dicounted coupons 
                for k in range(1, counter1 +1):  
                    PV_cpn.append(CPN * np.exp(-r *(itotal[-k] * dt - i * dt)))  # Append dicounted coupons in the list
                D[0] = sum(PV_cpn)  # Sum up the dicounted coupons 

            # Regular D[j] values
            # Exclude D[0], D[jmax]
            for j in range(1, jmax, 1):  
                D1 = -VT[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
                D2 = -VT[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
                D3 = -VT[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
                D[j] = D1+D2+D3

            ### Upper boundary condition in matrix terms
            A[jmax] = 0
            B[jmax] = 1
            C[jmax] = 0
            if (i+1) in ireview:
                counter2 += 1
            D[jmax] = (1000 + CPN) * np.exp(-r * (ireview[-counter2] * dt - i * dt))
            # ireview[-counter2]: time of next autocall

            ### Start LU decomposition
            alpha[0] = B[0]
            CN_S[0]  = D[0]

            # Where to stop solving?
            if i in ireview:         # Autocall dates
                StopFactor = j0+1    # Stop at j0 (Solve from 0~j0)
            else:                   
                StopFactor = jmax+1  # Stop at jmax (Solve from 0~jmax)

            # Solve alpha, CN_S
            for j in range(1, StopFactor, 1):
                alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
            VT[i,(StopFactor - 1)] = CN_S[(StopFactor - 1)]/alpha[(StopFactor - 1)]  

            # But at autocall dates, boundaries are different
            # A[j0] = 0, B[j0] = 1, C[j0] = 0, so change VT[i,j0:jmax+1] to D[j0]
            if i in ireview:           
                VT[i,j0:jmax+1] = (1000 + CPN) * np.exp(-r * (ipayment[ireview.index(i)] * dt - i * dt))
                # ipayment[ireview.index(i)]: time of coupon date

            # Start from j0-1 or jmax-1 to j0
            for j in range(StopFactor-2,-1,-1):
                VT[i,j] = (CN_S[j]-C[j]*VT[i,j+1])/alpha[j]
                if i in icpn:     # Payment dates for coupons T1,T2 (coupon only date) 
                    VT[i,j] += CPN
                if i in ireview:  # Payment dates for coupons T3~T17
                    VT[i,j] += CPN * np.exp(-r * (ipayment[ireview.index(i)] * dt - i * dt))      

        ### Grid for V (Trigger event does NOT happan) ###########################################################

        ### At maturity
        # Calculate option value at maturity V[imax,j]
        # Also calculate the probabilities A, B, C
        i = imax 

        for j in range(jb, jmax+1):  
            V[i, j] = (1000 + CPN) *  np.exp(-r *(TM - TFR))  
            A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
            B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
            C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j

        ### Backward in time
        counter3 = 0

        for i in range(imax-1, -1, -1):

            # Lower boundary condition for V
            A[jb] = 0
            B[jb] = 1
            C[jb] = 0

            # Get value from VT
            D[jb] = VT[i, jb] 

            # Regular D values
            for j in range(jb+1, jmax, 1):
                D1 = -V[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
                D2 = -V[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
                D3 = -V[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
                D[j] = D1+D2+D3

            # Upper boundary condition for V
            A[jmax] = 0
            B[jmax] = 1
            C[jmax] = 0
            if (i+1) in ireview:
                counter3 += 1
            D[jmax] = (1000 + CPN) * np.exp(-r * (ireview[-counter3] * dt - i * dt))
            # ireview[-counter3]: time of next autocall

            ### Start LU decomposition
            alpha[jb] = B[jb]
            CN_S[jb] = D[jb]

            # Where to stop solving?
            if i in ireview:         # Autocall dates
                StopFactor = j0+1    # Stop at j0 (Solve from 0~j0)
            else:
                StopFactor = jmax+1  # Stop at jmax (Solve from 0~jmax)

            # Solve alpha, CN_S
            for j in range(jb+1, StopFactor, 1):
                alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
            V[i,(StopFactor - 1)] = CN_S[(StopFactor - 1)]/alpha[(StopFactor - 1)]

            # But at autocall dates, boundaries are different
            # A[j0] = 0, B[j0] = 1, C[j0] = 0, so change V[i,j0:jmax+1] to D[j0]
            if i in ireview:          
                V[i,j0:jmax+1] = (1000 + CPN) * np.exp(-r * (ipayment[ireview.index(i)] * dt - i * dt))
                # ipayment[ireview.index(i)]: time of coupon date

            # Start from j0-1 or jmax-1
            for j in range(StopFactor-2,jb-1,-1):
                V[i,j] = (CN_S[j]-C[j]*V[i,j+1])/alpha[j]
                if i in icpn:      # Payment dates for coupons T1,T2 (coupon only date) 
                    V[i,j] += CPN
                if i in ireview:   # Payment dates for coupons T3~T17
                    V[i,j] += CPN * np.exp(-r * (ipayment[ireview.index(i)] * dt - i * dt))                      

        # If value lower than LBC(jb), then get value from VT
        V[:, 0:jb] = VT[:, 0:jb]
        
        # RELAY OUTPUTS TO DICTIONARY
        jcritreal = S0/dS
        jcrit = int(jcritreal)
        jcritB = int(Trigger/dS)+1
        Vcrit = V[0,jcrit]+ (S0 - jcrit * dS) / (dS) * (V[0,jcrit+1] - V[0,jcrit])
        Blambda = (jcritB*dS - Trigger)/dS
        output = {'S_steps': jmax, 't_steps': imax, 'CN': Vcrit, 'Barrier Lambda': Blambda}
        cnfdb_result.append(output)

    # return different j results
    return cnfdb_result

### Varying lambda

In [12]:
start_j = 50  # start j (jmaxmin)
end_j = 500   # end j   (jmaxmax)
j_step = 10   # j steps

In [13]:
cnfd_varyLamda = CNFD_1(S0, Trigger, TM, TFR, r, q, sigma, SU, imax, start_j, end_j, j_step, CPN, Review_dates, Payment_dates, CPN_payment_dates)

KeyboardInterrupt: 

In [ ]:
# CREATE A DATAFRAME FROM THE BINOMIAL MODEL OUTPUT
df_varyLamda = pd.DataFrame.from_dict(cnfd_varyLamda)

In [ ]:
df_varyLamda.head()

In [ ]:
df_varyLamda.tail()

In [ ]:
# EXPORT THE DATA TO A CSV FILE
# df_varyLamda.to_csv("/Users/changyushiuan/Desktop/2022Spring/FIN514/Project/Project2/cnfd.csv", index=False)

In [ ]:
plt.figure(figsize=(14,10))
plt.plot(df_varyLamda['S_steps'], df_varyLamda['CN'], '-', markersize=3)
plt.title("Option Value of CN")
plt.xlabel("Number of S steps")
plt.ylabel("Value")
# plt.savefig('/Users/changyushiuan/Desktop/2022Spring/FIN514/Project/Project1/cn_value.png')
plt.show()

### Lambda=1
In this case, since the continuous barrier (autocall) have the largest error, we can make the grid exactly on the notes,
by selecting lamda=1.

In [ ]:
start_j = 50  # start j (jmaxmin)
end_j = 500   # end j   (jmaxmax)
j_step = 25   # j steps

In [ ]:
cnfd_lamda1 = CNFD_1(S0, Trigger, TM, TFR, r, q, sigma, SU, imax, start_j, end_j, j_step, CPN, Review_dates, Payment_dates, CPN_payment_dates)

In [ ]:
# CREATE A DATAFRAME FROM THE BINOMIAL MODEL OUTPUT
df_Lamda1 = pd.DataFrame.from_dict(cnfd_lamda1)

In [ ]:
df_Lamda1.head()

In [ ]:
df_Lamda1.tail()

In [ ]:
plt.figure(figsize=(14,10))
plt.plot(df_Lamda1['S_steps'], df_Lamda1['CN'], '-', markersize=3)
plt.title("Option Value of CN")
plt.xlabel("Number of S steps")
plt.ylabel("Value")
# plt.savefig('/Users/changyushiuan/Desktop/2022Spring/FIN514/Project/Project1/cn_value.png')
plt.show()